In [ ]:
from arik_proxl import main,check_all
from bs4 import BeautifulSoup
import time
import concurrent.futures
from dictdiffer import diff, patch, swap, revert
import itertools
import requests
from bs4 import BeautifulSoup
import random
import concurrent.futures
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
from random import randint
import telegram
import copy
from colors import color, red, blue
from requests.exceptions import ConnectionError
import threading
from telegram import ParseMode
import datetime
from requests_ip_rotator import ApiGateway
import json 

class DictDiffer(object):
    """
    Calculate the difference between two dictionaries as:
    (1) items added
    (2) items removed
    (3) keys same in both but changed values
    (4) keys same in both and unchanged values
    """
    def __init__(self, current_dict, past_dict):
        self.current_dict, self.past_dict = current_dict, past_dict
        self.set_current, self.set_past = set(current_dict.keys()), set(past_dict.keys())
        self.intersect = self.set_current.intersection(self.set_past)
    def added(self):
        return self.set_current - self.intersect 
    def removed(self):
        return self.set_past - self.intersect 
    def changed(self):
        return set(o for o in self.intersect if self.past_dict[o] != self.current_dict[o])
    def unchanged(self):
        return set(o for o in self.intersect if self.past_dict[o] == self.current_dict[o])
#telegram bot
bot = telegram.Bot(token='1488268600:AAG-liK2MqXnmJ8uO6sWhtEUtUjo8tgi0vY')
#random user agent
software_names = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   
user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)
 


#loading proxies from arik_proxl function
a=main()
proxies=check_all(a)
f = open(r'C:\Users\arik\stock\url.json')    
prices = json.load(f)
URL=[]
for i in range(len(prices)):
    URL.append(prices[i]['stock'])
# proxy result could be None a function to handle that
def ra(proxy):
    while proxy==None:
      proxy = random.choice(proxies)  
    return proxy
#scraping function
def scrape_story_url(url):
  global URL
  #url=url.replace("https://","http://")
  user_agent_random =user_agent_rotator.get_random_user_agent()
  headerss={'User-Agent':'user_agent_random','Accept': 'application/json, text/plain, */*','Referer':'https://www.otcmarkets.com/'} 
  proxy = random.choice(proxies)
  proxy=ra(proxy)
  session = requests.Session()
  #session.mount("https://site.com", gateway)
  session.headers = headerss 
  proxiesS={'http':'http://'+proxy['ip']+':'+proxy['port']}
  session.proxies.update(proxiesS)
  try:
   #r = requests.get(url,headers=headerss,proxies={'http':'http://'+proxy['ip']+':'+proxy['port']})
   r= session.get(url)
  except:
   print("problem with request in {}".format(url))
   time.sleep(randint(14,20))
   return scrape_story_url(url)
  if r.status_code == 200:
    try:
       req=r.json()
       a={"outstanding":req,"url":url}
    except:
       print(url)
       time.sleep(randint(6,7))
       return scrape_story_url(url)
    if a==None:
     bad_url.append(url)
     print(url+'1234')
     time.sleep(randint(4,5))
     scrape_story_url(url)
    return a
  else:
     try:
      #print('error'+url)
      time.sleep(randint(5,6))
      return scrape_story_url(url)
     except:
      URL.remove(url)



        

#chunking to avoid errors
CHUNK_SIZE = 30
#multithreading
try:
     with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
        story_urls=URL
        iterator = iter(story_urls)
        results=[]
        futures = [
            executor.submit(scrape_story_url, url=url)
            for url in itertools.islice(iterator, CHUNK_SIZE)]
            
        while futures:
            done, futures = concurrent.futures.wait(
                futures, return_when=concurrent.futures.FIRST_COMPLETED
            )

            for task in done:
             result=task.result()
             results.append(task.result())
            for url in itertools.islice(iterator, len(done)):
                futures.add(executor.submit(scrape_story_url, url)) 
        results = sorted(results, key=lambda k: k['url'])
        prices=sorted(prices, key=lambda k: k['stock'])
        
        
       
except KeyboardInterrupt:
    pass